# Explicit Feedback Demo

This notebook demonstrates and measures explicit-feedback performance for the Torch-based algorithms.

## Setup

Load modules:

In [1]:
import sys
import logging
logging.basicConfig(stream=sys.stderr, level=logging.INFO)

In [2]:
import pandas as pd
import numpy as np

In [3]:
from lenskit import batch
from lenskit.metrics.predict import rmse
from lenskit.algorithms.bias import Bias
from lenskit.algorithms.als import BiasedMF
from lktorch.biasedmf import TorchBiasedMF

In [4]:
from tqdm.notebook import tqdm
tqdm.pandas()

## MovieLens 100K

In [5]:
train_100k = pd.read_parquet('data/ml-100k/train.parquet')
test_100k = pd.read_parquet('data/ml-100k/test.parquet')

### Bias Model

In [6]:
bias = Bias()
bias.fit(train_100k)

INFO:lenskit.algorithms.bias:building bias model for 99500 ratings
INFO:lenskit.algorithms.bias:global mean: 3.529
INFO:lenskit.algorithms.bias:computed means for 1682 items
INFO:lenskit.algorithms.bias:computed means for 943 users


In [7]:
bias_preds = batch.predict(bias, test_100k)
bias_preds

INFO:binpickle.write:pickled 1035 bytes with 6 buffers
INFO:lenskit.util.parallel:setting up ProcessPoolExecutor w/ 4 workers
INFO:lenskit.batch._predict:generating 500 predictions for 100 users (setup took  8ms)
INFO:lenskit.batch._predict:generated 500 predictions for 100 users in 3.06s


,user,item,rating,timestamp,prediction
0,2,255,4.0,888551341,3.516415
1,2,251,5.0,888552084,4.399913
2,2,312,3.0,888550631,3.320026
3,2,290,3.0,888551441,3.271258
4,2,315,1.0,888550774,4.288380
...,...,...,...,...,...
495,913,143,5.0,881725761,3.435030
496,913,95,4.0,880826766,3.469793
497,913,588,3.0,881449256,3.477411
498,913,185,4.0,881367173,3.764182


In [8]:
rmse(bias_preds['prediction'], bias_preds['rating'])

1.0516887

### ALSModel

In [9]:
als = BiasedMF(25)
als.fit(train_100k)

INFO:lenskit.util.debug:numba threading layer: tbb
INFO:lenskit.algorithms.als:[ 0ms] fitting bias model
INFO:lenskit.algorithms.bias:building bias model for 99500 ratings
INFO:lenskit.algorithms.bias:global mean: 3.529
INFO:lenskit.algorithms.bias:computed means for 1682 items
INFO:lenskit.algorithms.bias:computed means for 943 users
INFO:lenskit.algorithms.als:[ 23ms] normalizing ratings
INFO:lenskit.algorithms.als:[1.10s] training biased MF model with ALS for 25 features
INFO:lenskit.algorithms.als:[4.13s] finished epoch 0 (|ΔP|=40.418, |ΔQ|=48.175)
INFO:lenskit.algorithms.als:[4.13s] finished epoch 1 (|ΔP|=15.190, |ΔQ|=21.709)
INFO:lenskit.algorithms.als:[4.14s] finished epoch 2 (|ΔP|=9.746, |ΔQ|=12.388)
INFO:lenskit.algorithms.als:[4.14s] finished epoch 3 (|ΔP|=6.599, |ΔQ|=8.389)
INFO:lenskit.algorithms.als:[4.15s] finished epoch 4 (|ΔP|=4.832, |ΔQ|=6.181)
INFO:lenskit.algorithms.als:[4.16s] finished epoch 5 (|ΔP|=3.747, |ΔQ|=4.759)
INFO:lenskit.algorithms.als:[4.16s] finished epo

In [10]:
als_preds = batch.predict(als, test_100k)
als_preds

INFO:binpickle.write:pickled 1592 bytes with 10 buffers
INFO:lenskit.util.parallel:setting up ProcessPoolExecutor w/ 4 workers
INFO:lenskit.batch._predict:generating 500 predictions for 100 users (setup took  5ms)
INFO:lenskit.batch._predict:generated 500 predictions for 100 users in 3.99s


,user,item,rating,timestamp,prediction
0,2,255,4.0,888551341,3.077152
1,2,251,5.0,888552084,4.552990
2,2,312,3.0,888550631,3.370210
3,2,290,3.0,888551441,3.197553
4,2,315,1.0,888550774,4.247648
...,...,...,...,...,...
495,913,143,5.0,881725761,3.802119
496,913,95,4.0,880826766,3.363786
497,913,588,3.0,881449256,3.305021
498,913,185,4.0,881367173,3.822973


In [11]:
rmse(als_preds['prediction'], als_preds['rating'])

0.9992593300711662

### Biased MF

In [12]:
tmf = TorchBiasedMF(25, epochs=20)
tmf.fit(train_100k)

INFO:lktorch.biasedmf:[ 0ms] preparing input data set
INFO:lktorch.biasedmf:preparing to train on cuda


epoch:   0%|          | 0/20 [00:00<?, ?it/s]

INFO:lktorch.biasedmf:[ 261ms] beginning epoch 1
INFO:lktorch.data.batch:re-shuffling 99500 rows


  0%|          | 0/13 [00:00<?, ?it/s]

INFO:lktorch.biasedmf:[1.85s] epoch 1 finished (|P|=7.681, |Q|=10.294, b=3.528)
INFO:lktorch.biasedmf:[1.86s] beginning epoch 2
INFO:lktorch.data.batch:re-shuffling 99500 rows


...

INFO:lktorch.biasedmf:[3.56s] epoch 18 finished (|P|=16.866, |Q|=20.862, b=3.479)
INFO:lktorch.biasedmf:[3.56s] beginning epoch 19
INFO:lktorch.data.batch:re-shuffling 99500 rows


  0%|          | 0/13 [00:00<?, ?it/s]

INFO:lktorch.biasedmf:[3.66s] epoch 19 finished (|P|=17.811, |Q|=21.943, b=3.474)
INFO:lktorch.biasedmf:[3.66s] beginning epoch 20
INFO:lktorch.data.batch:re-shuffling 99500 rows


  0%|          | 0/13 [00:00<?, ?it/s]

INFO:lktorch.biasedmf:[3.77s] epoch 20 finished (|P|=18.745, |Q|=23.013, b=3.468)
INFO:lktorch.biasedmf:finished training


In [13]:
tmf_preds = batch.predict(tmf, test_100k)
tmf_preds

INFO:binpickle.write:pickled 704 bytes with 7 buffers
INFO:lenskit.util.parallel:setting up ProcessPoolExecutor w/ 4 workers
INFO:lenskit.batch._predict:generating 500 predictions for 100 users (setup took  6ms)
INFO:lenskit.batch._predict:generated 500 predictions for 100 users in 3.11s


,user,item,rating,timestamp,prediction
0,2,255,4.0,888551341,3.371031
1,2,251,5.0,888552084,3.894303
2,2,312,3.0,888550631,3.322607
3,2,290,3.0,888551441,3.360593
4,2,315,1.0,888550774,3.826373
...,...,...,...,...,...
495,913,143,5.0,881725761,3.419306
496,913,95,4.0,880826766,3.652405
497,913,588,3.0,881449256,3.535137
498,913,185,4.0,881367173,3.905253


In [14]:
rmse(tmf_preds['prediction'], tmf_preds['rating'])

1.0534589